Импорт библиотек

In [ ]:
!pip install torch torchvision
!pip install easyocr
import easyocr
import PIL
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
from PIL import Image
import cv2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 16.2 MB/s eta 0:00:00


In [ ]:
reader = easyocr.Reader(['en','ru'], gpu = True)

Функция

In [ ]:
def Check_separator(path):
  bounds = reader.readtext(path)
  im = Image.open(path)
  dots_coner, dots_sered = [],[]
  for bound in bounds: #сохранение двух точек бокса
    p0, p1, p2, p3 = bound[0]
    dots_coner.append(p0)
    dots_coner.append(p1)
    dot = [(p0[0] + p2[0])/2, (p0[1] + p2[1])/2]
    dots_sered.append(dot)
  k_ans, projection_dot = 0, []
  for j in dots_coner: #прорекция на прямую
    y = (k_ans * j[0] - k_ans*k_ans*j[1])/(1 - k_ans*k_ans)
    if k_ans == 0:
      x = j[0]
    else:
      x = y/k_ans
    dot = [x, y]
    projection_dot.append(dot)
  x, g, right, left, perep = [], 0, [], [], [] + [0]
  for i in projection_dot:
    x = x + [i[0]]
  x.sort()
  l, r = x[len(x)-1], l/len(x)
  for i in range(0, l, 15): #подсчет количества точек возле прямой
    rcount = 0
    lcount = 0
    for j in x:
      if (j < i+15) and (j >= i - 15):
        lcount = lcount + 1
    left = left + [lcount]
  for i in range(len(left)): #нахождение самых больших скоплений точек
    if left[i] >= 17:
      perep = perep + [i]
  X, count. gr, dotss = np.array(dots_coner), 0, [], x
  x, y = im.size
  perep = perep + [x/15]
  for i in range(len(perep)): #фильтрация
    if i != 0:
      if perep[i] - perep[i - 1] < 5:
        count = count + 1
      else:
        gr = gr +  [(perep[i  - 1] + perep[i - 1 - count])/2]
        count = 0
    if i == len(perep) - 1:
      gr = gr + [(perep[i] + perep[i - count])/2]
  corp_imgs = []
  for i in range(1, len(gr)):  #обрезка изображений
    corp_imgs = corp_imgs + [im.crop((gr[i-1]*15, 0, gr[i]*15, y))]
  return corp_imgs, gr

In [ ]:
def Check_separator_CV(path):
  bounds = reader.readtext(path)
  im = cv2.imread(path)

  dots_coner = []
  dots_sered = []

  for bound in bounds: #сохранение двух точек бокса
    p0, p1, p2, p3 = bound[0]
    dots_coner.append(p0)
    dots_coner.append(p1)


  k_ans = 0
  projection_dot = []

  for j in dots_coner: #прорекция на прямую
    y = (k_ans * j[0] - k_ans*k_ans*j[1])/(1 - k_ans*k_ans)
    if k_ans == 0:
      x = j[0]
    else:
      x = y/k_ans
    dot = [x, y]
    projection_dot.append(dot)

  x = []
  g = 0
  right = []
  left = []
  perep = [] + [0]

  for i in projection_dot:
    x = x + [i[0]]

  x.sort()

  l = x[len(x)-1]
  r = l/len(x)

  for i in range(0, l, 15): #подсчет количества точек возле прямой
    lcount = 0
    for j in x:
      if (j < i+15) and (j >= i - 15):
        lcount = lcount + 1
    left = left + [lcount]

  for i in range(len(left)): #нахождение самых больших скоплений точек
    if left[i] >= 17:
      perep = perep + [i]

  X = np.array(dots_coner)
  count = 0
  gr = []

  x, y = im.shape[1], im.shape[0]
  perep = perep + [x/15]
  for i in range(len(perep)): #фильтрация
    if i != 0:
      if perep[i] - perep[i - 1] < 5:
        count = count + 1
      else:
        gr = gr +  [(perep[i  - 1] + perep[i - 1 - count])/2]
        count = 0
    if i == len(perep) - 1:
      gr = gr + [(perep[i] + perep[i - count])/2]

  corp_imgs = []
  for i in range(1, len(gr)): #обрезка
    corp_imgs.append(im[0:y, int(gr[i-1]*15):int(gr[i]*15)])
  return corp_imgs

In [ ]:
bounds = reader.readtext('/content/1615325560_57-p-roza-chek-meit-obraz-60.jpg')

error: ignored

In [ ]:
a = Check_separator('/content/LkAAAgKYYOA-1920.jpg')

In [ ]:
print(cv2.getBuildInformation())


General configuration for OpenCV 4.8.0 =====================================
  Version control:               4.8.0-dirty

  Extra modules:
    Location (extra):            /io/opencv_contrib/modules
    Version control (extra):     4.8.0

  Platform:
    Timestamp:                   2023-08-09T11:58:06Z
    Host:                        Linux 5.15.0-1042-azure x86_64
    CMake:                       3.27.1
    CMake generator:             Unix Makefiles
    CMake build tool:            /bin/gmake
    Configuration:               Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2 AVX512_SKX
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (16 files):         + SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      FP16 (0 files):            + SSSE3 SSE4_1 POPCNT SSE4_2 FP16 AVX
      AVX (7 files):      

In [ ]:
Check_separator('/content/951efe5s-1920.jpg')

([<PIL.Image.Image image mode=RGB size=352x1080>,
  <PIL.Image.Image image mode=RGB size=270x1080>,
  <PIL.Image.Image image mode=RGB size=82x1080>,
  <PIL.Image.Image image mode=RGB size=346x1080>,
  <PIL.Image.Image image mode=RGB size=494x1080>,
  <PIL.Image.Image image mode=RGB size=353x1080>],
 [0.5, 24.0, 42.0, 47.5, 70.5, 103.5, 127.0])

In [ ]:
Check_separator('/content/Screenshot_1.jpg')

SyntaxError: ignored